In [ ]:
# Only needs to be run upon first-time use
import sys
!{sys.executable} -m pip install alpha_vantage

In [8]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import requests
# https://github.com/RomelTorres/alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators 
from alpha_vantage.sectorperformance import SectorPerformances

In [9]:
API_URL = 'https://www.alphavantage.co/query?'
API_KEY = 'TMANNBF5NO707ZEI'

In [11]:
# Output formats: json dictionaries (default), DataFrame, csv.
# csv does not support ForeignExchange, SectorPerformances, TechIndicators.
# indexing_type can also be integer (default: date)
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='date')
ti = TechIndicators(key=API_KEY, output_format='pandas')
sp = SectorPerformances(key=API_KEY, output_format='pandas')

In [12]:
# symbols = list of stock tickers as strings
# intraday_interval: '1min', '5min', '15min', '30min', '60min' (default '15min')
def get_data(symbol, start_date= None, end_date= None, type='daily_adjusted', interval= None):
    
    if type=='intraday':
        stock_data, stock_meta_data = ts.get_intraday(symbol, interval=interval, outputsize='full')
    elif type=='daily':
        stock_data, stock_meta_data = ts.get_daily(symbol, outputsize='full')
    elif type=='daily_adjusted':
        stock_data, stock_meta_data = ts.get_daily_adjusted(symbol, outputsize='full')
    elif type=='weekly':
        stock_data, stock_meta_data = ts.get_weekly(symbol)
    elif type=='weekly_adjusted':
        stock_data, stock_meta_data = ts.get_weekly_adjusted(symbol)
    elif type=='monthly':
        stock_data, stock_meta_data = ts.get_monthly(symbol)
    elif type=='monthly_adjusted':
        stock_data, stock_meta_data = ts.get_monthly_adjusted(symbol)
    
    stock_data.rename(columns=lambda col: col[3:], inplace=True)
    
    if start_date and end_date != None:
        stock_data = stock_data.loc[start_date:end_date]
    elif start_date != None:
        stock_data = stock_data.loc[start_date:]
    elif end_date != None:
        stock_data = stock_data.loc[:end_date]
    return stock_data

In [13]:
def calculate_returns(stock_data):
    
    daily_close = stock_data['close']
    daily_pct_change = daily_close.pct_change()
    daily_pct_change.fillna(0, inplace=True)
    
    daily_log_returns = np.log(daily_close.pct_change()+1)
    daily_log_returns.fillna(0, inplace=True)
    
    # Cumulative daily rate of return is useful to determine the value of an investment at regular intervals.
    cum_daily_return = (1 + daily_pct_change).cumprod()
   # cum_monthly_return = cum_daily_return.resample('M').mean()
    
    return pd.DataFrame({'pct change': daily_pct_change,
                        'log returns':daily_log_returns,
                        'cumulative daily':cum_daily_return})